In [2]:
import pandas as pd
import yfinance as yf

In [3]:
def safe_get(df, row, col):
    try:
        return df.loc[row, col]
    except KeyError:
        return 0
    except:
        return 0

In [4]:
def get_financial_ratios(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)

    # Fetch financial statements
    bs = stock.balance_sheet  # Balance Sheet
    is_ = stock.financials    # Income Statement
    cf = stock.cashflow       # Cash Flow Statement
    info = stock.info         # Market data

    try:
        # Use latest columns (most recent fiscal year)
        year = bs.columns[0]

        # Extracting necessary values with safe fallbacks
        current_assets = safe_get(bs, 'Total Current Assets', year)
        current_liabilities = safe_get(bs, 'Total Current Liabilities', year)
        inventory = safe_get(bs, 'Inventory', year)
        cash = safe_get(bs, 'Cash', year) + safe_get(bs, 'Cash And Cash Equivalents', year)
        total_liabilities = safe_get(bs, 'Total Liab', year)
        total_assets = safe_get(bs, 'Total Assets', year)
        total_equity = safe_get(bs, 'Total Stockholder Equity', year)
        short_term_debt = safe_get(bs, 'Short Long Term Debt', year) + safe_get(bs, 'Short Term Debt', year)
        long_term_debt = safe_get(bs, 'Long Term Debt', year)
        total_debt = short_term_debt + long_term_debt

        net_income = safe_get(is_, 'Net Income', year)
        ebit = safe_get(is_, 'Ebit', year)
        interest_expense = safe_get(is_, 'Interest Expense', year)
        gross_profit = safe_get(is_, 'Gross Profit', year)
        operating_income = safe_get(is_, 'Operating Income', year)
        revenue = safe_get(is_, 'Total Revenue', year)
        cogs = safe_get(is_, 'Cost Of Revenue', year)
        receivables = safe_get(bs, 'Net Receivables', year)

        eps = info.get('trailingEps', 0)
        pe_ratio = info.get('trailingPE', 0)
        pb_ratio = info.get('priceToBook', 0)
        dividend_yield = info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0
        dividend_rate = info.get('dividendRate', 0)

        # Liquidity Ratios
        current_ratio = current_assets / current_liabilities if current_liabilities else 0
        quick_ratio = (current_assets - inventory) / current_liabilities if current_liabilities else 0
        cash_ratio = cash / current_liabilities if current_liabilities else 0

        # Solvency (Leverage) Ratios
        debt_to_equity = total_debt / total_equity if total_equity else 0
        interest_coverage = ebit / interest_expense if interest_expense else float('inf')
        debt_ratio = total_liabilities / total_assets if total_assets else 0

        # Profitability Ratios
        net_profit_margin = net_income / revenue if revenue else 0
        gross_profit_margin = gross_profit / revenue if revenue else 0
        operating_margin = operating_income / revenue if revenue else 0
        return_on_assets = net_income / total_assets if total_assets else 0
        return_on_equity = net_income / total_equity if total_equity else 0

        # Efficiency Ratios (Activity)
        asset_turnover = revenue / total_assets if total_assets else 0
        inventory_turnover = cogs / inventory if inventory else float('inf')
        receivables_turnover = revenue / receivables if receivables else float('inf')
        dso = 365 / receivables_turnover if receivables_turnover else float('inf')
        dio = 365 / inventory_turnover if inventory_turnover else float('inf')

        # Market Valuation Ratios
        dividend_payout_ratio = (dividend_rate / eps) if eps else 0

        return {
            'Liquidity Ratios': {
                'Current Ratio': current_ratio,
                'Quick Ratio': quick_ratio,
                'Cash Ratio': cash_ratio
            },
            'Solvency Ratios': {
                'Debt-to-Equity Ratio': debt_to_equity,
                'Interest Coverage Ratio': interest_coverage,
                'Debt Ratio': debt_ratio
            },
            'Profitability Ratios': {
                'Net Profit Margin': net_profit_margin,
                'Gross Profit Margin': gross_profit_margin,
                'Operating Profit Margin': operating_margin,
                'Return on Assets (ROA)': return_on_assets,
                'Return on Equity (ROE)': return_on_equity
            },
            'Efficiency Ratios': {
                'Asset Turnover Ratio': asset_turnover,
                'Inventory Turnover Ratio': inventory_turnover,
                'Receivables Turnover Ratio': receivables_turnover,
                'Days Sales Outstanding (DSO)': dso,
                'Days Inventory Outstanding (DIO)': dio
            },
            'Market Valuation Ratios': {
                'P/E Ratio': pe_ratio,
                'P/B Ratio': pb_ratio,
                'EPS': eps,
                'Dividend Yield (%)': dividend_yield,
                'Dividend Payout Ratio': dividend_payout_ratio
            }
        }
    except Exception as e:
        print("Error fetching or computing data:", e)
        return None


In [5]:
def save_ratios_to_csv(ratios, ticker_symbol, filename="financial_ratios.csv"):
    if ratios:
        all_data = {'Ticker': [ticker_symbol]}
        for category, metrics in ratios.items():
            for name, value in metrics.items():
                all_data[f'{category} - {name}'] = [value]

        df = pd.DataFrame(all_data)
        df.to_csv(filename, index=False)
        print(f"Financial ratios for {ticker_symbol} saved to {filename}")
    else:
        print(f"No ratios to save for {ticker_symbol}.")


In [6]:
ticker = "RELIANCE.NS"
ratios = get_financial_ratios(ticker)
if ratios:
    for category, metrics in ratios.items():
        print(f"\n{category}:")
        for name, value in metrics.items():
            print(f"  {name}: {value:.2f}")
    save_ratios_to_csv(ratios, ticker, f"{ticker}_financial_ratios.csv")


Liquidity Ratios:
  Current Ratio: 0.00
  Quick Ratio: 0.00
  Cash Ratio: 0.00

Solvency Ratios:
  Debt-to-Equity Ratio: 0.00
  Interest Coverage Ratio: 0.00
  Debt Ratio: 0.00

Profitability Ratios:
  Net Profit Margin: 0.08
  Gross Profit Margin: 0.25
  Operating Profit Margin: 0.12
  Return on Assets (ROA): 0.04
  Return on Equity (ROE): 0.00

Efficiency Ratios:
  Asset Turnover Ratio: 0.51
  Inventory Turnover Ratio: 4.42
  Receivables Turnover Ratio: inf
  Days Sales Outstanding (DSO): 0.00
  Days Inventory Outstanding (DIO): 82.66

Market Valuation Ratios:
  P/E Ratio: 23.09
  P/B Ratio: 1.95
  EPS: 51.17
  Dividend Yield (%): 42.00
  Dividend Payout Ratio: 0.10
Financial ratios for RELIANCE.NS saved to RELIANCE.NS_financial_ratios.csv


In [7]:
tickers = ["TCS.NS", "INFY.NS", "HDFCBANK.NS"]
for ticker in tickers:
    ratios = get_financial_ratios(ticker)
    if ratios:
        save_ratios_to_csv(ratios, ticker, f"{ticker}_financial_ratios.csv")

Financial ratios for TCS.NS saved to TCS.NS_financial_ratios.csv
Financial ratios for INFY.NS saved to INFY.NS_financial_ratios.csv
Financial ratios for HDFCBANK.NS saved to HDFCBANK.NS_financial_ratios.csv


In [8]:
sectors_tickers = {
    "Banking": ["HDFCBANK.NS", "ICICIBANK.NS", "SBIN.NS"],
    "IT": ["TCS.NS", "INFY.NS", "TECHM.NS"]
}

output_dir = "financial_ratios_by_sector"
import os
os.makedirs(output_dir, exist_ok=True)

for sector, tickers in sectors_tickers.items():
    all_sector_ratios = []
    for ticker in tickers:
        print(f"\nFetching ratios for {ticker} ({sector})...")
        ratios = get_financial_ratios(ticker)
        if ratios:
            all_data = {'Ticker': [ticker]}
            for category, metrics in ratios.items():
                for name, value in metrics.items():
                    all_data[f'{category} - {name}'] = [value]
            all_sector_ratios.append(pd.DataFrame(all_data))

    if all_sector_ratios:
        combined_df = pd.concat(all_sector_ratios, ignore_index=True)
        filename = os.path.join(output_dir, f"{sector}_financial_ratios.csv")
        combined_df.to_csv(filename, index=False)
        print(f"\nFinancial ratios for {sector} sector saved to {filename}")
    else:
        print(f"\nNo financial ratios to save for {sector} sector.")


Fetching ratios for HDFCBANK.NS (Banking)...

Fetching ratios for ICICIBANK.NS (Banking)...

Fetching ratios for SBIN.NS (Banking)...

Financial ratios for Banking sector saved to financial_ratios_by_sector\Banking_financial_ratios.csv

Fetching ratios for TCS.NS (IT)...

Fetching ratios for INFY.NS (IT)...

Fetching ratios for TECHM.NS (IT)...

Financial ratios for IT sector saved to financial_ratios_by_sector\IT_financial_ratios.csv
